### 1. Prepare Workspace

In [1]:
!pip install transformers torch &> /dev/null

In [2]:
!pip install -U langchain &>/dev/null

In [3]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
import random
from IPython.display import display, Markdown
from langchain.prompts import ChatPromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [4]:
# Load tokenizer and model
checkpoint = "Qwen/Qwen2.5-Coder-7B-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [5]:
# Function to process the chat with mode selection for reasoning
def generate_response(system_message, user_message):
    conversation = f"{system_message.content}\nUser: {user_message.content}\nAI:"

    # Tokenize input and create attention mask
    tokens = tokenizer(conversation, return_tensors="pt")
    inputs = tokens['input_ids'].to(device)
    attention_mask = tokens['attention_mask'].to(device)

    # Generate AI's response
    temperature = 0.1
    max_new_tokens = 300

    # produce a response from the AI model
    outputs = model.generate(inputs,
                             attention_mask=attention_mask,
                             max_new_tokens=max_new_tokens,
                             temperature=temperature,
                             top_p=0.9, do_sample=True)

    # Decode and return the AIMessage
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = output_text.split('AI:')[-1].strip()
    return AIMessage(content=response_text)


### 2. Connection on SQLite Database

In [6]:
# connection to SQLite database
conn = sqlite3.connect('marketing.db')
cursor = conn.cursor()

### 3. Building tables

In [7]:
cursor.execute('''
DROP TABLE IF EXISTS customers;
''')
# Build customers table
cursor.execute('''
CREATE TABLE customers (
    customerID INTEGER PRIMARY KEY,
    city TEXT,
    occupation TEXT,
    age INTEGER
);
''')

cursor.execute('''
DROP TABLE IF EXISTS campaigns;
''')
# Build campaigns table
cursor.execute('''
CREATE TABLE campaigns (
    campaignID INTEGER PRIMARY KEY,
    product TEXT,
    start_date DATE,
    end_date DATE,
    budget NUMERIC,
    channel TEXT
);
''')

cursor.execute('''
DROP TABLE IF EXISTS sales;
''')
# Build sales table
cursor.execute('''
CREATE TABLE sales (
    saleID INTEGER PRIMARY KEY,
    customerID INTEGER,
    campaignID INTEGER,
    product TEXT,
    quantity INTEGER,
    sale_date DATE,
    saleamount NUMERIC
);
''')

conn.commit()

In [8]:
pd.read_sql_query("SELECT * FROM customers;", conn)

,customerID,city,occupation,age


In [9]:
pd.read_sql_query("SELECT * FROM campaigns;", conn)

,campaignID,product,start_date,end_date,budget,channel


In [10]:
pd.read_sql_query("SELECT * FROM sales;", conn)

,saleID,customerID,campaignID,product,quantity,sale_date,saleamount


### Input Data

In [11]:
random.seed(0)

# Define customer data
customerIDs = list(range(1000, 1100))
cities = ["NEW YORK", "CHICAGO", "LOS ANGELES", "PHILADELPHIA"]
occupations = ["ENGINEER", "DOCTOR", "LAWYER", "TEACHER"]
ages = range(25, 60)

customers = []
for i in customerIDs:
    city = random.choice(cities)
    occupation = random.choice(occupations)
    age = random.choice(ages)
    customers.append((i, city, occupation, age))

# Insert data into the customers table
cursor.executemany('''
INSERT INTO customers (customerID, city, occupation, age)
VALUES (?, ?, ?, ?)
''', customers)
conn.commit()



In [12]:
pd.read_sql_query("SELECT *  FROM customers;", conn)

,customerID,city,occupation,age
0,1000,PHILADELPHIA,TEACHER,27
1,1001,LOS ANGELES,TEACHER,50
2,1002,LOS ANGELES,TEACHER,47
3,1003,CHICAGO,DOCTOR,43
4,1004,CHICAGO,ENGINEER,41
...,...,...,...,...
95,1095,LOS ANGELES,LAWYER,51
96,1096,NEW YORK,ENGINEER,55
97,1097,PHILADELPHIA,LAWYER,46
98,1098,NEW YORK,TEACHER,32


In [13]:
# Define campaigns data
campaignIDs = list(range(1, 101))
products = ["BIKE", "TENT", "KAYAK", "TREADMILL"]
channels = ["EMAILS", "TV", "SOCIAL NETWORK", "RADIO"]
budgets = range(1000, 50000, 1000)

campaigns = []
for i in campaignIDs:
    channel = random.choice(channels)
    product = random.choice(products)
    budget = random.choice(budgets)

    start_date = datetime(2020, 1, 1) + timedelta(
        days=random.randint(0, (datetime(2024, 12, 31) - datetime(2020, 1, 1)).days))
    end_date = start_date + timedelta(days=90)

    campaigns.append((i, product, start_date.date(), end_date.date(), budget, channel))

# Insert data into the campaigns table
cursor.executemany('''
INSERT INTO campaigns (campaignID, product, start_date, end_date, budget, channel)
VALUES (?, ?, ?, ?, ?, ?)
''', campaigns)
conn.commit()


In [14]:
pd.read_sql_query("SELECT *  FROM campaigns;", conn)

,campaignID,product,start_date,end_date,budget,channel
0,1,KAYAK,2023-11-08,2024-02-06,48000,SOCIAL NETWORK
1,2,TENT,2023-03-02,2023-05-31,41000,TV
2,3,BIKE,2024-07-12,2024-10-10,5000,RADIO
3,4,TENT,2021-03-28,2021-06-26,48000,EMAILS
4,5,TREADMILL,2020-07-19,2020-10-17,1000,EMAILS
...,...,...,...,...,...,...
95,96,BIKE,2024-12-14,2025-03-14,18000,EMAILS
96,97,BIKE,2022-03-26,2022-06-24,17000,EMAILS
97,98,TREADMILL,2024-03-08,2024-06-06,7000,RADIO
98,99,KAYAK,2024-11-19,2025-02-17,19000,SOCIAL NETWORK


In [16]:
# Retrieve customer IDs
cursor.execute('SELECT customerID FROM customers')
customer_ids = [row[0] for row in cursor.fetchall()]

# Retrieve campaigns with their details
cursor.execute('SELECT campaignID, product, start_date, end_date FROM campaigns')
campaigns = cursor.fetchall()

# Parameters
num_sales = 100  # Total number of sales records to generate
quantity_range = (1, 10)  # Quantity per sale

# Define unit prices for products
unit_prices = {
    "BIKE": 300,
    "TENT": 150,
    "KAYAK": 400,
    "TREADMILL": 700
}

sales = []
sale_id_start = 1  # Starting ID for sales

for sale_id in range(sale_id_start, sale_id_start + num_sales):
    # Select a random campaign
    campaign = random.choice(campaigns)
    campaign_id, product, start_date_str, end_date_str = campaign

    # Convert string dates to datetime objects
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

    # Generate a random sale date within the campaign period
    delta_days = (end_date - start_date).days
    random_days = random.randint(0, delta_days) if delta_days > 0 else 0
    sale_date = start_date + timedelta(days=random_days)

    # Select a random customer
    customer_id = random.choice(customer_ids)

    # Determine quantity and calculate sale amount
    quantity = random.randint(*quantity_range)
    sale_amount = quantity * unit_prices.get(product, 100)  # Default unit price if not specified

    # Create the sales record
    sales.append((
        sale_id,
        customer_id,
        campaign_id,
        product,
        quantity,
        sale_date.date(),
        sale_amount
    ))

# Insert data into the sales table
cursor.executemany('''
    INSERT INTO sales (saleID, customerID, campaignID, product, quantity, sale_date, saleamount)
    VALUES (?, ?, ?, ?, ?, ?, ?)
''', sales)

# Commit the transaction
conn.commit()


In [17]:
pd.read_sql_query("SELECT * FROM sales", conn)

,saleID,customerID,campaignID,product,quantity,sale_date,saleamount
0,1,1068,34,KAYAK,6,2022-04-05,2400
1,2,1031,16,TENT,3,2022-06-17,450
2,3,1037,9,TREADMILL,5,2022-02-26,3500
3,4,1073,67,TENT,9,2021-03-19,1350
4,5,1068,81,TREADMILL,2,2020-02-13,1400
...,...,...,...,...,...,...,...
95,96,1084,86,TREADMILL,4,2021-08-14,2800
96,97,1085,34,KAYAK,3,2022-04-10,1200
97,98,1045,40,TREADMILL,10,2022-07-30,7000
98,99,1093,70,BIKE,3,2023-04-11,900


### Prompt Engineering for join SQL query

In [18]:
# Define initial messages with prompt engineering
system_message = SystemMessage(content="You are a senior data analyst. Give an help in coding with SQL")
user_message = HumanMessage(content="""

  Please write an SQL query to connect the three tables according to the rules provided.

  Tables:

  1. customers (customerID INTEGER PRIMARY KEY, city TEXT, occupation TEXT, age INTEGER)
  2. campaigns (campaignID INTEGER PRIMARY KEY, product TEXT, start_date DATE, end_date DATE, budget NUMERIC, channel TEXT)
  3. sales (saleID INTEGER PRIMARY KEY, customerID INTEGER, campaignID INTEGER, product TEXT, quantity INTEGER, sale_date DATE, saleamount NUMERIC)

  Rules:
  Retrive all fields from the tables.

  SQL Query:
  """)

# Generate response
sql_query = generate_response(system_message, user_message)
print(sql_query.content)

Certainly! To retrieve all fields from the three tables (`customers`, `campaigns`, and `sales`), you need to perform a join operation based on the relationships between these tables. Typically, the `sales` table has foreign keys that reference the primary keys of the `customers` and `campaigns` tables.

Here's how you can write the SQL query:

```sql
SELECT 
    c.customerID,
    c.city,
    c.occupation,
    c.age,
    ca.campaignID,
    ca.product AS campaign_product,
    ca.start_date,
    ca.end_date,
    ca.budget,
    ca.channel,
    s.saleID,
    s.quantity,
    s.sale_date,
    s.saleamount
FROM 
    customers c
JOIN 
    sales s ON c.customerID = s.customerID
JOIN 
    campaigns ca ON s.campaignID = ca.campaignID;
```

### Explanation:
- **SELECT Clause**: This part specifies the columns you want to retrieve. I've used aliases (`c`, `ca`, `s`) for the tables to make the query more readable.
- **FROM Clause**: The main table is `customers`, aliased as `c`.
- **JOIN Clauses**:
 

### Run SQL join query



In [19]:
pd.read_sql_query(
'''
SELECT
    c.customerID,
    c.city,
    c.occupation,
    c.age,
    ca.campaignID,
    ca.product AS campaign_product,
    ca.start_date,
    ca.end_date,
    ca.budget,
    ca.channel,
    s.saleID,
    s.quantity,
    s.sale_date,
    s.saleamount
FROM
    customers c
JOIN
    sales s ON c.customerID = s.customerID
JOIN
    campaigns ca ON s.campaignID = ca.campaignID;
''', conn)

,customerID,city,occupation,age,campaignID,campaign_product,start_date,end_date,budget,channel,saleID,quantity,sale_date,saleamount
0,1068,LOS ANGELES,TEACHER,28,34,KAYAK,2022-02-25,2022-05-26,27000,SOCIAL NETWORK,1,6,2022-04-05,2400
1,1031,LOS ANGELES,ENGINEER,39,16,TENT,2022-04-11,2022-07-10,37000,RADIO,2,3,2022-06-17,450
2,1037,NEW YORK,LAWYER,32,9,TREADMILL,2022-01-04,2022-04-04,13000,TV,3,5,2022-02-26,3500
3,1073,NEW YORK,DOCTOR,32,67,TENT,2021-03-02,2021-05-31,2000,SOCIAL NETWORK,4,9,2021-03-19,1350
4,1068,LOS ANGELES,TEACHER,28,81,TREADMILL,2020-01-18,2020-04-17,27000,EMAILS,5,2,2020-02-13,1400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1084,LOS ANGELES,DOCTOR,38,86,TREADMILL,2021-05-22,2021-08-20,12000,RADIO,96,4,2021-08-14,2800
96,1085,PHILADELPHIA,LAWYER,25,34,KAYAK,2022-02-25,2022-05-26,27000,SOCIAL NETWORK,97,3,2022-04-10,1200
97,1045,PHILADELPHIA,ENGINEER,31,40,TREADMILL,2022-07-28,2022-10-26,25000,EMAILS,98,10,2022-07-30,7000
98,1093,NEW YORK,LAWYER,36,70,BIKE,2023-04-04,2023-07-03,35000,EMAILS,99,3,2023-04-11,900


### Prompt Engineering for group by SQL query

In [20]:
# Define initial messages with prompt engineering
system_message = SystemMessage(content="You are a senior data analyst. Give an help in coding with SQL")
user_message = HumanMessage(content="""

  Please write an SQL query according to the rules provided.

  Tables:

  1. customers (customerID INTEGER PRIMARY KEY, city TEXT, occupation TEXT, age INTEGER)
  2. campaigns (campaignID INTEGER PRIMARY KEY, product TEXT, start_date DATE, end_date DATE, budget NUMERIC, channel TEXT)
  3. sales (saleID INTEGER PRIMARY KEY, customerID INTEGER, campaignID INTEGER, product TEXT, quantity INTEGER, sale_date DATE, saleamount NUMERIC)

  Rules:
  Join all the tables. Group by total of saleamount per city and channel. The fields to show city, channel and sum of saleamount.

  SQL Query:
  """)

# Generate response
sql_query = generate_response(system_message, user_message)
print(sql_query.content)

To achieve the task described, you need to join the `customers`, `sales`, and `campaigns` tables based on their respective keys. Then, group the results by `city` and `channel` and calculate the sum of `saleamount`. Here is the SQL query that accomplishes this:

```sql
SELECT 
    c.city,
    ca.channel,
    SUM(s.saleamount) AS total_saleamount
FROM 
    customers c
JOIN 
    sales s ON c.customerID = s.customerID
JOIN 
    campaigns ca ON s.campaignID = ca.campaignID
GROUP BY 
    c.city, ca.channel;
```

### Explanation:
- **Joining Tables**:
  - `customers c`: Alias for the `customers` table.
  - `sales s ON c.customerID = s.customerID`: Joins the `sales` table with the `customers` table using the `customerID`.
  - `campaigns ca ON s.campaignID = ca.campaignID`: Joins the `sales` table with the `campaigns` table using the `campaignID`.

- **Selecting Fields**:
  - `c.city`: Selects the city from the `customers` table.
  - `ca.channel`: Selects the channel from the `campaigns` table

### Run SQL group by query

In [21]:
pd.read_sql_query(
'''
SELECT
    c.city,
    ca.channel,
    SUM(s.saleamount) AS total_saleamount
FROM
    customers c
JOIN
    sales s ON c.customerID = s.customerID
JOIN
    campaigns ca ON s.campaignID = ca.campaignID
GROUP BY
    c.city, ca.channel;
''', conn)

,city,channel,total_saleamount
0,CHICAGO,EMAILS,14400
1,CHICAGO,RADIO,22800
2,CHICAGO,SOCIAL NETWORK,10050
3,CHICAGO,TV,11600
4,LOS ANGELES,EMAILS,17200
5,LOS ANGELES,RADIO,13050
6,LOS ANGELES,SOCIAL NETWORK,12050
7,LOS ANGELES,TV,4100
8,NEW YORK,EMAILS,4500
9,NEW YORK,RADIO,5850


### Prompt Engineering for group by and filter SQL query

In [22]:
# Define initial messages with prompt engineering
system_message = SystemMessage(content="You are a senior data analyst. Give an help in coding with SQL")
user_message = HumanMessage(content="""

  Please write an SQL query according to the rules provided.

  Tables:

  1. customers (customerID INTEGER PRIMARY KEY, city TEXT, occupation TEXT, age INTEGER)
  2. campaigns (campaignID INTEGER PRIMARY KEY, product TEXT, start_date DATE, end_date DATE, budget NUMERIC, channel TEXT)
  3. sales (saleID INTEGER PRIMARY KEY, customerID INTEGER, campaignID INTEGER, product TEXT, quantity INTEGER, sale_date DATE, saleamount NUMERIC)

  Rules:
  Join all the tables. Group by total of saleamount per city and channel. The fields to show are channel, city and total saleamount. Filter by "SOCIAL NETWORK" channel

  SQL Query:
  """)

# Generate response
sql_query = generate_response(system_message, user_message)
print(sql_query.content)

To achieve the desired result, you need to join the `customers`, `sales`, and `campaigns` tables based on their respective keys. Then, group the results by `channel`, `city`, and calculate the total `saleamount`. Finally, filter the results to include only those where the `channel` is 'SOCIAL NETWORK'. Here's the SQL query that accomplishes this:

```sql
SELECT 
    c.channel,
    cu.city,
    SUM(s.saleamount) AS total_saleamount
FROM 
    customers cu
JOIN 
    sales s ON cu.customerID = s.customerID
JOIN 
    campaigns c ON s.campaignID = c.campaignID
WHERE 
    c.channel = 'SOCIAL NETWORK'
GROUP BY 
    c.channel, cu.city;
```

Explanation:
- **Joining Tables**: The `JOIN` operations connect the `customers`, `sales`, and `campaigns` tables based on their primary keys (`customerID` and `campaignID`).
- **Filtering**: The `WHERE` clause filters the records to include only those from the 'SOCIAL NETWORK' channel.
- **Grouping and Aggregation**: The `GROUP BY` clause groups the results

### Run group by and filter SQL query

In [23]:
pd.read_sql_query(
    '''
SELECT
    c.channel,
    cu.city,
    SUM(s.saleamount) AS total_saleamount
FROM
    customers cu
JOIN
    sales s ON cu.customerID = s.customerID
JOIN
    campaigns c ON s.campaignID = c.campaignID
WHERE
    c.channel = 'SOCIAL NETWORK'
GROUP BY
    c.channel, cu.city;
    ''',conn)

,channel,city,total_saleamount
0,SOCIAL NETWORK,CHICAGO,10050
1,SOCIAL NETWORK,LOS ANGELES,12050
2,SOCIAL NETWORK,NEW YORK,12450
3,SOCIAL NETWORK,PHILADELPHIA,12400


In [24]:
cursor.close()
conn.close()

### Riferimenti

In [ ]:
# https://huggingface.co/Qwen/Qwen2.5-Coder-7B-Instruct
# https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html
# https://python.langchain.com/api_reference/core/messages/langchain_core.messages.human.HumanMessage.html
# https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html
# https://python.langchain.com/api_reference/core/messages/langchain_core.messages.system.SystemMessage.html